# Pytorch Tutorial

Working with data, creating models, optimizing model parameters, and saving the trained models

## Quickstart

### Working with data

torch.utils.data.DataLoader and torch.utils.data.Dataset
Dataset stores the samples and corresponding labels, DataLoader wraps an iterable around Dataset.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets.

Torchvision.datasets

In [2]:
# Download training data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



We pass Dataset as an argument to DataLoader.
This wraps an iterable over the dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.
Here we define a batch size of 64 - each element in the dataloader iterable will return a batch of 64 features and labels.

In [3]:
batch_size = 64

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}{y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])torch.int64


### Creating models
To define a nn in PyTorch, we create a class that inherits from nn.Module.
We define the layers of the network in __init__ function, and specify how data will pass through the network in the forward function.

In [5]:
# Get cpu/gpu for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimize model parametrs
To train a model, we need a loss function and an optimizer

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the trainign dataset (fed to it in batches), and backpropagates the prediction error to adjest the model's parameters

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch  % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}[{current:>5d}|{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuraxy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The trainign process is conducted over several iterations (epochs). During each epoch, the model leans parameters to make better prediction. We print accuracy change with each epoch.

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.317179[   64|60000]
loss: 2.300580[ 6464|60000]
loss: 2.276749[12864|60000]
loss: 2.266305[19264|60000]
loss: 2.268577[25664|60000]
loss: 2.232506[32064|60000]
loss: 2.241753[38464|60000]
loss: 2.216914[44864|60000]
loss: 2.207024[51264|60000]
loss: 2.170359[57664|60000]
Test Error: 
 Accuraxy: 42.0%, Avg loss: 2.171621 

Epoch 2
-------------------------------
loss: 2.187252[   64|60000]
loss: 2.177525[ 6464|60000]
loss: 2.119441[12864|60000]
loss: 2.129583[19264|60000]
loss: 2.108339[25664|60000]
loss: 2.036381[32064|60000]
loss: 2.069111[38464|60000]
loss: 2.003845[44864|60000]
loss: 1.999834[51264|60000]
loss: 1.926460[57664|60000]
Test Error: 
 Accuraxy: 57.7%, Avg loss: 1.931031 

Epoch 3
-------------------------------
loss: 1.969463[   64|60000]
loss: 1.941880[ 6464|60000]
loss: 1.828668[12864|60000]
loss: 1.855707[19264|60000]
loss: 1.783568[25664|60000]
loss: 1.712988[32064|60000]
loss: 1.735969[38464|60000]
loss: 1.646079[44864

### Saving models
A common way to save a model is to serialize the internal state dictioonary (containing the model parameters)

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading models
Recreating the model structure and loading the state dictionary into it

In [12]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

# This model can now be used to make predictions

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


## Tensors